### Install and load necessary packages

In [5]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

### Load dataset then extract the 'Issue' column and convert list to single string

In [8]:
complaints_df = pd.read_csv("C:/Users/charl/Documents/Complaint Analysis/rows.csv.zip",
                           low_memory=False)

issues = complaints_df['Issue'].dropna().tolist()
text = ' '.join(issues)